In today's notes, we are going to focus on the following: 

1. Strassen's Matrix Multiplication algorithm. 
2. Optimizing the implementation of the algorithm in Julia. 
3. Paralellizing the algorithm in Julia while keeping the efficiecy. 
4. Benchmark and show that the algorithm is indeed having a complexity less than $O(N^3)$, assuming the matrices are $N \times N$. 

---
### Strassen's Matrix Multiplication Algorithm

The objective of the algorithm is to compute the product of 2 matrices: $A$, $B$. The algorithm uses an recursive approach, here we consider the recursive structure of the matrix in the following ways: 

$$
A = \begin{bmatrix}
A_{1, 1} & A_{1, 2} \\
A_{2, 1} & A_{2, 2} 
\end{bmatrix}
\quad 
B = \begin{bmatrix}
B_{1, 1} & B_{1, 2}\\
B_{2, 1} & B_{2, 2}
\end{bmatrix}
\quad 
C = \begin{bmatrix}
C_{1, 1} & C_{1, 2}\\
C_{2, 1} & C_{2, 2}
\end{bmatrix}
$$

And then, we use these submatrices to compute some intermediate matrices:
$$M_1 = (A_{1, 1}+ A_{2,2})(B_{1, 1} + B_{2,2})$$
$$M_2 = (A_{2, 1} + A_{2,2})B_{1,1}$$
$$M_3 = A_{1,1}(B_{1,2} - B_{2,2})$$
$$M_4 = A_{2,2}(B_{2,1} - B_{1,1})$$
$$M_5 = (A_{1,1} + A_{1,2})B_{2,2}$$
$$M_6 = (A_{2,1} - A_{1,1})(B_{1,1} + B_{1,2})$$
$$M_7 = (A_{1,2} - A_{2,2})(B_{2,1} + B_{2,2})$$

Using these 7 intermediate matrices which are computed using only 7 multiplications, we will get sub-matrices for the $C$ matrix:

$$
C_{1, 1} = M_1 + M_4 - M_5 + M_7
$$
$$
C_{1, 2} = M_3 + M_5
$$
$$
C_{2, 1} = M_2 + M_4
$$
$$
C_{2,2} = M_1 - M_2 + M_3 + M_6
$$





### Extra Rows or Columns that cannot be partitioned
The matrices need to be equally partitioned, meaning that they have size equals to $2^N$. Is it possible to bypass this? 

#### Considering Remainder of Partitioning
Let $A$ be $m\times n$ and $B$ be $n \times k$. Consider the case where $m, n, k$ are all not divisible by 2, then after partitioning the submatrices, we take the last row and column of the matrices as the remainding rows/columns. Then the matrices can be structurally represented as the following: 

$$
A = \begin{bmatrix}
    \widehat{A} & b \\
    a^T & x
\end{bmatrix} 
\quad 
B = \begin{bmatrix}
    \widehat{B} & c \\
    d^T & y
\end{bmatrix}
$$

Then, conveniently, we can write the produce $AB$ to be: 

$$
\begin{bmatrix}
    \widehat{A} & b \\
    a^T & x
\end{bmatrix} 
\begin{bmatrix}
    \widehat{B} & c \\
    d^T & y
\end{bmatrix}
=\begin{bmatrix}
\widehat{A}\widehat{B} + bd^T & \widehat{A}c + by \\ 
a^T\widehat{B} + xd^T & a^Tc + xy
\end{bmatrix}
$$

And the only sub-matrices that require the Strassen's multiplications is the sub-matrix at top left corner. 

When any of the extra vectors, $a, b, c, d$ are not present, then it can be treated as zero vector, and then we need to shrink the boundary of the resulting matrix into its sub-matrix located at the top left corner. 

### More implementation Notes

1. Garbage collection time. 
2. Multi-threading.
3. DFS compute tree or BFS compute tree? 
    3.1. For DFS, single thread can achieve without much use of GC. 
    3.2. For BFS, a buttom up algorithm can ahieve without much use of GC with multi-threading. (**Hard to Implement, needs topolotical sorting on the compute tree**)

In [16]:
A = rand(3, 3); B = rand(3, 3)
display(A)
display(B)
C = view(A, 1:2, 1:2) + view(B, 1:2, 1:2)
display(C)
display(A)
display(B)
typeof(C)
println("")
typeof(view(A, 1:2, 1:2))


3×3 Array{Float64,2}:
 0.406059    0.28012   0.417667
 0.786407    0.490074  0.757986
 0.00143017  0.232561  0.289402

3×3 Array{Float64,2}:
 0.534398  0.295209  0.77953
 0.625349  0.291282  0.180967
 0.861469  0.84314   0.682705

2×2 Array{Float64,2}:
 0.940457  0.575329
 1.41176   0.781356

3×3 Array{Float64,2}:
 0.406059    0.28012   0.417667
 0.786407    0.490074  0.757986
 0.00143017  0.232561  0.289402

3×3 Array{Float64,2}:
 0.534398  0.295209  0.77953
 0.625349  0.291282  0.180967
 0.861469  0.84314   0.682705

SubArray{Float64,2,Array{Float64,2},Tuple{UnitRange{Int64},UnitRange{Int64}},false}

In [11]:
Arr = Array{Array{Float64}}(undef, 0)
append!(Arr, [zeros(4,4)])

1-element Array{Array{Float64,N} where N,1}:
 [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0]